In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

In [2]:
!pip install --upgrade pip

In [3]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 703.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 26.2 

In [4]:
# !pip install bert-extractive-summarizer

In [5]:
from autogluon.multimodal import MultiModalPredictor
from autogluon.core.utils.loaders import load_pd

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [6]:
path='../input/court-data/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
train_facts=pd.read_csv(path+'ai_trans_train_facts.csv')
test_facts=pd.read_csv(path+'ai_trans_test_facts.csv')

In [7]:
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

In [8]:
train['facts']=train_facts['facts']
test['facts']=test_facts['facts']

In [9]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    text = re.sub(r'\s+', ' ', text)  # 공백 축소

    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]  # 불용어 제거

    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]  # 어간 추출

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]  # 표제어 추출

    text = ' '.join(words)
    return text


In [11]:
train["first_party"] = train["first_party"].apply(preprocess_text)
test["first_party"] = test["first_party"].apply(preprocess_text)
print("first_party done")
train["second_party"] = train["second_party"].apply(preprocess_text)
test["second_party"] = test["second_party"].apply(preprocess_text)
print("second_party done")

first_party done
second_party done


In [12]:
train

,first_party,second_party,facts,first_party_winner
0,phil st amant,herman thompson,june phil st amant candid public offic made te...,1
1,stephen duncan,lawrenc owen,ramon nelson ride bike suffer lethal blow back...,0
2,billi joe magwood,toni patterson warden et al,billi joe magwood murder sentenc death sentenc...,1
3,linklett,walker,victor linklett convict state court evid illeg...,0
4,william earl fike,alabama,april selma alabama intrud broke apart daughte...,1
...,...,...,...,...
2473,hollyfronti cheyenn refin llc et al,renew fuel associ et al,congress amend clean air act energi polici act...,1
2474,grupo mexicano de desarrollo,allianc bond fund inc,allianc bond fund inc invest fund purchas appr...,1
2475,peguero,unit state,peguero right appeal sentenc later motion habe...,0
2476,immigr natur servic,st cyr,charg sell control substanc convict made depor...,0


In [13]:
predictor = MultiModalPredictor(label='first_party_winner',eval_metric='acc').fit(train_data=train,presets="best_quality",
                                                                                  time_limit=3600*10)
# predictor = MultiModalPredictor(label='first_party_winner',eval_metric='acc').fit(train_data=train,verbosity=3,time_limit=2000)

No path specified. Models will be saved in: "AutogluonModels/ag-20230629_101159/"
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
AutoMM starts to create your model. ✨

- AutoGluon version is 0.8.0.

- Pytorch version is 1.13.1+cu117.

- Model will be saved to "/kaggle/working/AutogluonModels/ag-20230629_101159".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /kaggle/working/AutogluonModels/ag-20230629_101159
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/autogluon/multimodal/utils/environment.py:59: UserWarning: Interactive environment is detected. Currently, MultiModalPredictor does not support multi-gpu training under an interactive environment due to the limitation of ddp / ddp_spawn strategies in PT Lightning. Thus, we switch to single gpu training. For multi-gpu training, you need to execute MultiModalPredictor in a script.
  warnings.warn(
2 GPUs are detected, and 1 GPUs will be used.
   - GPU 0 name: 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/kaggle/working/AutogluonModels/ag-20230629_101159")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /kaggle/working/AutogluonModels/ag-20230629_101159
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




In [14]:
# predictions = predictor.predict(test)

In [15]:
predictions = predictor.predict_proba(test)
predictions 

Predicting: 0it [00:00, ?it/s]

,0,1
0,0.281900,0.718100
1,0.315630,0.684370
2,0.309545,0.690455
3,0.347205,0.652795
4,0.317532,0.682468
...,...,...
1235,0.311951,0.688049
1236,0.317850,0.682150
1237,0.309858,0.690142
1238,0.292797,0.707203


In [16]:
predictions=predictions.iloc[: , 1]

In [17]:
print((predictions >= 0.5).sum())

1240


In [18]:
limit=0.7
print((predictions >= limit).sum())

241


In [19]:
predictions07 = predictions.apply(lambda x: 1 if x >= limit else 0)

In [20]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["first_party_winner"]=predictions07
submission.to_csv('/kaggle/working/submission_advance_07.csv', index=False)

In [21]:
value_counts = submission["first_party_winner"].value_counts()
if 0 in value_counts.index:
    zero_count = value_counts[0]
    print("Number of zeroes in 'first_party_winner':", zero_count)
else:
    print("Zero count not found in 'first_party_winner'")

Number of zeroes in 'first_party_winner': 999


In [22]:
predictions06 = predictions.apply(lambda x: 1 if x >= 0.6 else 0)

In [23]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["first_party_winner"]=predictions06
submission.to_csv('/kaggle/working/submission_advance_06.csv', index=False)

In [24]:
value_counts = submission["first_party_winner"].value_counts()
if 0 in value_counts.index:
    zero_count = value_counts[0]
    print("Number of zeroes in 'first_party_winner':", zero_count)
else:
    print("Zero count not found in 'first_party_winner'")

Zero count not found in 'first_party_winner'


In [25]:
# def preprocess_text(text):
#     text = text.lower()  # 소문자로 변환
#     text = re.sub(r'\d+', '', text)  # 숫자 제거
#     text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
#     text = re.sub(r'\s+', ' ', text)  # 공백 축소

#     words = text.split()
#     stop_words = set(stopwords.words('english'))
#     words = [word for word in words if word not in stop_words]  # 불용어 제거

#     stemmer = PorterStemmer()
#     words = [stemmer.stem(word) for word in words]  # 어간 추출

#     lemmatizer = WordNetLemmatizer()
#     words = [lemmatizer.lemmatize(word) for word in words]  # 표제어 추출

#     text = ' '.join(words)
#     return text


In [26]:
# train["facts"] = train["facts"].apply(preprocess_text)
# test["facts"] = test["facts"].apply(preprocess_text)
# print("facts done")
# train["first_party"] = train["first_party"].apply(preprocess_text)
# test["first_party"] = test["first_party"].apply(preprocess_text)
# print("first_party done")
# train["second_party"] = train["second_party"].apply(preprocess_text)
# test["second_party"] = test["second_party"].apply(preprocess_text)
# print("second_party done")

In [27]:
# import torch
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5ForConditionalGeneration.from_pretrained("t5-small")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Model will run on {device}")

# model.to(device)  # 모델을 GPU로 옮김

# def ai_summarize(text):
#     inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=6000, truncation=True)
#     inputs = inputs.to(device)  # 입력 텐서를 GPU로 옮김
#     outputs = model.generate(inputs, max_length=500, min_length=10, length_penalty=2.0, num_return_sequences=1, early_stopping=True)
#     summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return summary


In [28]:
# train["facts"] = train["facts"].apply(ai_summarize)
# print("done")
# test["facts"] = test["facts"].apply(ai_summarize)
# print("done")

In [29]:
# from transformers import LongformerTokenizer, LongformerModel

# # Longformer 모델과 토크나이저 불러오기
# model_name = 'allenai/longformer-base-4096'
# tokenizer = LongformerTokenizer.from_pretrained(model_name)
# model = LongformerModel.from_pretrained(model_name)

# # 최대 길이 설정
# max_length = 500

# # train 데이터와 test 데이터의 'facts' 컬럼에 대해 요약 수행
# train['summary'] = train['facts'].apply(lambda x: tokenizer.decode(tokenizer.encode(x, truncation=True, max_length=max_length, return_tensors='pt')[0], skip_special_tokens=True))
# test['summary'] = test['facts'].apply(lambda x: tokenizer.decode(tokenizer.encode(x, truncation=True, max_length=max_length, return_tensors='pt')[0], skip_special_tokens=True))

# # 요약 결과 출력
# print("Train 데이터의 요약:")
# print(train['summary'])
# print("\nTest 데이터의 요약:")
# print(test['summary'])

In [30]:
# # 'summary' 컬럼의 문자열 길이에 대한 기본 통계를 계산합니다.
# train_summary_lengths = train['summary'].apply(len)

# print(f'Average length: {train_summary_lengths.mean()}')
# print(f'Maximum length: {train_summary_lengths.max()}')
# print(f'Minimum length: {train_summary_lengths.min()}')

In [31]:
# # 'fact' 컬럼의 문자열 길이에 대한 기본 통계를 계산합니다.
# train_fact_lengths = train['facts'].apply(len)

# print(f'Average length: {train_fact_lengths.mean()}')
# print(f'Maximum length: {train_fact_lengths.max()}')
# print(f'Minimum length: {train_fact_lengths.min()}')

In [32]:
# from summarizer import Summarizer

# # BERT 모델 로드
# model = Summarizer()

# # `fact` 컬럼의 각 행을 요약 (train 데이터)
# train['summary'] = train['summary'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=500) != '' else x)

# # `fact` 컬럼의 각 행을 요약 (test 데이터)
# test['summary'] = test['summary'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=500) != '' else x)

In [33]:
# from summarizer import Summarizer

# # BERT 모델 로드
# model = Summarizer()

# # `fact` 컬럼의 각 행을 요약 (train 데이터)
# train['facts'] = train['facts'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=500) != '' else x)

# # `fact` 컬럼의 각 행을 요약 (test 데이터)
# test['facts'] = test['facts'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=500) != '' else x)


In [34]:
# # 'fact' 컬럼의 문자열 길이에 대한 기본 통계를 계산합니다.
# train_fact_lengths = train['facts'].apply(len)

# print(f'Average length: {train_fact_lengths.mean()}')
# print(f'Maximum length: {train_fact_lengths.max()}')
# print(f'Minimum length: {train_fact_lengths.min()}')

In [35]:
# import re
# from nltk.tokenize import word_tokenize

# # 텍스트 정제 함수
# def clean_text(text):
#     text = re.sub('[^a-zA-Z]', ' ', text)  # 특수 문자 제거
#     text = text.lower()  # 모두 소문자로 변환
#     text = text.strip()  # 불필요한 공백 제거
#     return text

# # 토큰화 함수
# def tokenize_text(text):
#     return word_tokenize(text)

# # 정제 및 토큰화를 적용할 컬럼
# columns_to_clean_and_tokenize = ['first_party', 'second_party', 'facts']

# # 'train' 및 'test' 데이터셋의 각 컬럼에 함수 적용
# for column in columns_to_clean_and_tokenize:
# #     train[column] = train[column].apply(clean_text).apply(tokenize_text)
# #     test[column] = test[column].apply(clean_text).apply(tokenize_text)
#     train[column] = train[column].apply(clean_text)
#     test[column] = test[column].apply(clean_text)